In [ ]:
import requests
from bs4 import BeautifulSoup 
import re
import unicodedata
import pythainlp.util
from pythainlp.tokenize import word_tokenize
from pythainlp.util import find_keyword
from pythainlp.util import rank
#from pythainlp.summarize import extract_keywords
from pythainlp.summarize import summarize
import itertools
import sqlite3
from urllib.parse import urljoin
import time
from pythainlp.tag import tag_provinces


In [ ]:
def crawl(url, depth,visited):
    if depth < 2 :
        visited.add(url)
        headers = {'User-Agent': "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/42.0.2311.135 Safari/537.36 Edge/12.246"}
        #time.sleep(0.25)
        response = requests.get(url,headers=headers)
        try:
            soup = BeautifulSoup(response.text, 'html.parser')
        except:
            soup = BeautifulSoup(response.text, 'lxml')
        links = soup.find_all('a')
        links = [link.get('href') for link in links if link.get('href') and not link.get('href').startswith('#')]
        links = [urljoin(url, link) for link in links if link]

        for link in links:
            if link not in visited:
                visited.add(link)
                #if link.startswith(url):
                crawl(link,depth=depth+1, visited=visited)
    return visited
base_url = 'https://news.kapook.com'
website_dict3 = crawl(base_url, 0, set())
print(website_dict3)
print(len(website_dict3))

In [ ]:
def check_domain(base_url,links):
    result= set()
    for link in links :
        if link.startswith(base_url):
            result.add(link)
    return result



domain = check_domain(base_url,website_dict3)
print(len(domain))

In [ ]:
def check_not_domain(base_url,links):
    result= set()
    for link in links :
        if not link.startswith(base_url):
            result.add(link)
    return result

not_domain = check_not_domain(base_url,website_dict3)
print(not_domain)

In [ ]:
target_links= {'https://www.thairath.co.th':0,'https://news.kapook.com':0}
def check_ref(base,links):
    for i in links:
        for j in base:
            if i.startswith(j):
                target_links[base_url]+=1
    return target_links

ref=check_ref(target_links,not_domain)
print(ref)
                

In [ ]:
class Thai:
    def __init__(self,data:list):
        self.data_value = data
        self.sentence = self.get_sentence()
        self.keyword = self.get_keyword()
        self.summarize = self.get_summarize()
    def make_sentence(self,list_word):
        self.sentence_value = ''
        for i in list_word:
            for i in list_word:
                if pythainlp.util.countthai(i)<30:
                    list_word.remove(i)
        self.sentence_value = ' '.join(list_word)
        return self.sentence_value
    def get_sentence(self):
        self.sentence_result = self.make_sentence(self.data_value)
        self.sentence_result = unicodedata.normalize("NFKD", self.sentence_result)
        return self.sentence_result
    def get_keyword(self):
        self.keyword_result = {}
        self.keyword_value = self.get_tokenize()
        try:
            self.keyword_dict = find_keyword(self.keyword_value)
        # Iterate over the keys in the dictionary
            for key in self.keyword_dict:
        # Check if the key is text (i.e., not a space or quotation mark)
                if key.isalpha():
            # If the key is text, add it to the new dictionary
                    self.keyword_result[key] = self.keyword_dict[key]
            return self.keyword_result
        except:
             self.keyword_result['NONE'] = 0
             return self.keyword_result
        
    def get_summarize(self):
        self.summarize_result =[]
        self.summarize_result = summarize(self.sentence,n=10)
        return self.summarize_result
    def get_tokenize(self):
        self.word_token = word_tokenize(self.sentence, engine="newmm")
        return self.word_token
    def get_location(self):
        self.location_Result = self.location()
        self.location_counts = {}
        for location, _ in self.location_Result:
            if location in self.location_counts:
                self.location_counts[location] += 1
            else:
                self.location_counts[location] = 1
        return self.location_counts
    def location(self):
        self.data = self.get_tokenize()
        self.location_value = tag_provinces(self.data)
        self.Result_location = filtered_input = [entry for entry in self.location_value if entry[1] == 'B-LOCATION']
        return self.Result_location

In [ ]:
def scrape_tags(url):
  response = requests.get(url)
  soup = BeautifulSoup(response.text, 'html.parser')
  title_tag = soup.find('title').text
  p_tags = soup.find_all('p')
  p_list =[]
  for p in p_tags:
    if p.string != None:
      p_list.append(unicodedata.normalize("NFKD", p.string))
  if len(p_list) == 0:
    p_list.append('ไม่พบข้อความในเว็บนี้')
  if  title_tag == None:
    title_tag = 'None'
  p_tag = "".join(p_list)
    
  return p_tag, title_tag,p_list

In [ ]:
def scrape_tags(url):
  response = requests.get(url)
  soup = BeautifulSoup(response.text, 'html.parser')
  title_tag = soup.find('title').text
  p_tags = soup.find_all('p')
  p_list =[]

  return p_tags
  
url =  'https://www.thairath.co.th/news/foreign/2628056'
scrape_tags(url)

In [ ]:
def scrape_tags(url):
  response = requests.get(url)
  soup = BeautifulSoup(response.text, 'html.parser')
  title_tag = soup.find('title').text
  p_tags = soup.find_all('p')
  p_list =[]
  for p in p_tags:
    if p.string != None:
      p_list.append(unicodedata.normalize("NFKD", p.string))
  if len(p_list) == 0:
    p_list.append('ไม่พบข้อความในเว็บนี้')
  
  p_tag = "".join(p_list)
  thai_nlp = Thai(p_list)
  keyword = thai_nlp.keyword
  keyword = {k: v for k, v in sorted(keyword.items(), key=lambda item: item[1], reverse=True)}
  keyword = dict(itertools.islice(keyword.items(), 5))
  summarize_article = thai_nlp.summarize
  try:
    location = 'จ.'+max(thai_nlp.get_location().keys())
  except:
    location = 'NONE'
  
  if  title_tag == None:
    title_tag = summarize_article[0]
    
  return p_tag, title_tag, keyword,location

In [ ]:
d=dict()
for i in domain:
    doc = []
    p_tag, title_tag, keyword,location = scrape_tags(i)
    doc.append(i)
    doc.append(p_tag)
    doc.append(title_tag)
    doc.append(location)
    lk = list(keyword)
    for i in lk:
        if i not in d:
            d[i]=list()
        d[i].extend(doc)
print(d)

In [ ]:
def scrape_tags(url):
  response = requests.get(url)
  soup = BeautifulSoup(response.text, 'html.parser')
  title_tag = soup.find('title').text
  p_tags = soup.find_all('p')
  p_list =[]
  for p in p_tags:
    if p.string != None:
      p_list.append(unicodedata.normalize("NFKD", p.string))
  if len(p_list) == 0:
    p_list.append('ไม่พบข้อความในเว็บนี้')
  
  p_tag = "".join(p_list)
  thai_nlp = Thai(p_list)
  keyword = thai_nlp.keyword
  keyword = {k: v for k, v in sorted(keyword.items(), key=lambda item: item[1], reverse=True)}
  keyword = dict(itertools.islice(keyword.items(), 5))
  summarize_article = thai_nlp.summarize
  try:
    location = 'จ.'+max(thai_nlp.get_location().keys())
  except:
    location = 'NONE'
  
  if  title_tag == None:
    title_tag = summarize_article[0]
    
  return p_tag, title_tag, keyword,location

def insert_data_to_db(website, p_tag, title, keyword,location):
  
  for i in keyword:
    conn.execute("INSERT INTO DATA5 (WEBSITE, BODY, TITLE, KEYWORD, WORD_FREQUENCY ,LOCATION) VALUES (?, ?, ?, ?, ?, ?)", (website, p_tag, title, i, keyword[i],location))
    conn.commit()
    print(f'For website {website}')
    print(f'For the p tags is:{p_tag}') 
    print(f'For the title tag is: {title}')
    print(f'For the keyword is:{i}')
    print(f'Forthe word frequency is:{keyword[i]}')
    print(f'For the location is:{location}')

 

# Iterate through the websites
conn = sqlite3.connect('scraped_data.db')
conn.execute('''CREATE TABLE DATA5
             (ID INTEGER PRIMARY KEY AUTOINCREMENT,
             WEBSITE STRING NOT NULL,
             BODY TEXT NOT NULL,
             TITLE TEXT NOT NULL,
             KEYWORD TEXT NOT NULL,
             WORD_FREQUENCY INT NOT NULL,
             LOCATION TEXT );''')
j=0
for website in domain:
  p_tag, title, keyword,location = scrape_tags(website)
  insert_data_to_db(website, p_tag, title, keyword,location)
  j+=1
  print(j)
       


conn.close()



